In [ ]:
import os
import getpass
import subprocess

class AoaClient(): 
    
    def __init__(self, username="admin", password="admin"):
        # aoa username & password
        self.username = username
        self.password = password
        
        # add AOA cli tool to path
        os.environ["PATH"] += os.pathsep + "../../../../../cli/"
        
    def configure_vantage_test_credentials(self): 
        os.environ["TD_USERNAME"] = input("Username: ")
        os.environ["TD_PASSWORD"] = getpass.getpass("Password:")
        
    def add_dataset(self, name, metadata, desc=""): 
        # can upload to AOA via API (validate name unique)
        self._write_json("models/" + name + ".json", metadata)
        
    def set_model_config(self, metadata): 
        self._write_json("../../config.json", metadata)
        
    def train(self, dataset):
        subprocess.check_output("run-model-cli.py 78036940-248e-4dae-b690-6f70eefcb188 evaluate -d models/{}.json".format(dataset), shell=True)
        
    def evaluate(self, dataset):
        subprocess.check_output("run-model-cli.py 78036940-248e-4dae-b690-6f70eefcb188 evaluate -d models/{}.json".format(dataset), shell=True)
    
    def _write_json(self, filename, obj):
        with open(filename, "w") as f:
            json.dump(obj, f)

In [ ]:

cli = AoaClient()

cli.configure_vantage_test_credentials()

## Dataset metadata
# build the dataset metadata which **must** also exist in the AOA 
cli.add_dataset("train", metadata={
    "hostname": "tddb-env-d-268.vantage.demo.intellicloud.teradata.com",
    "schema": "your-schema",
    "model_table": "csi_telco_churn_model",
    "data_table": "model_dataset_train"
})


# write the evaluation dataset metadata
cli.add_dataset("train", metadata={
    "hostname": "tddb-env-d-268.vantage.demo.intellicloud.teradata.com",
    "schema": "your-schema",
    "model_table": "csi_telco_churn_model",
    "data_table": ".model_dataset_test",
    "results_table": "telco_stat_output"
})

    
## Model configuration 
# Notice how the model_type hyperparameters is the name we use in the ModelType argument for the MLE
cli.set_model_config(metadata={
      "hyperParameters": {
          "type": "Naive Bayes",
          "model_type": "Bernoulli"
      }
    })
